# Tugas 1 (Crawling Data)

*   Pendapat Masyarakat tentang Ganjar Pranowo yang maju sebagai calon presiden periode 2024 - 2029 (Twitter)
*   Portal Tugas Akhir Universitas Trunojoyo
*   Berita Online

## Persiapan Environment

### Instalasi Library SNSCRAPE

In [2]:
!pip3 install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 5.2 MB/s eta 0:00:00


### Instalasi BeautifulSoup

In [3]:
!pip install requests
!pip install html5lib
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

### Pembuatan Fungsi untuk Analisis Sentimen

In [ ]:
from textblob import TextBlob
import re
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
  analysis = TextBlob(clean_tweet(tweet))
  if analysis.sentiment.polarity > 0:
    return 1
  elif analysis.sentiment.polarity == 0:
    return 0
  else:
    return -1

  def analize_subjectorobject(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
      return 'Positif'
    elif analysis.sentiment.polarity == 0:
      return 'Neutral'
    else:
      return 'Negatif'
    

## Crawling Data dari Twitter dan Konversi Hasil Data ke CSV

In [ ]:
tweets = []

In [ ]:
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Ganjar Pranowo Calon Presiden since:2022-01-01 until:2023-02-20').get_items()):
    if i>500:
        break
    tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])

# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets, columns=['Datetime', 'Username Twitter', 'Tweet', 'Label', 'Lokasi'])

<ipython-input-19-ef96f4d4f9f2>:5: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])


In [ ]:
tweets_df['Label'] = tweets_df['Label'].replace([0,1,-1], ['Netral', 'Positif', 'Negatif'])

In [ ]:
tweets_df

,Datetime,Username Twitter,Tweet,Label,Lokasi
0,2023-02-19 13:45:59+00:00,GustiMberkahi,LAMPUNG PUNYA CERITA\n\nRelawan Ganjar Pranowo...,Netral,Yogya 9/9/2020
1,2023-02-18 09:38:55+00:00,2045Tv,Semakin jelas tanda-tanda Megawati memilih Gan...,Netral,
2,2023-02-17 13:59:33+00:00,terkinidotid,Jokowi Mania Soal Ide Duet Prabowo dan Ganjar:...,Netral,"Makassar, Indonesia"
3,2023-02-17 12:30:41+00:00,RonaldiesYafet,"Ketika mantan relawan Jokowi, ubah haluan mend...",Netral,Indonesia
4,2023-02-17 09:50:04+00:00,amarillo_solido,Koalisi Indonesia Bersatu (KIB) memunculkan na...,Netral,
...,...,...,...,...,...
496,2022-12-25 15:26:39+00:00,sutrvey,"Pendiri lembaga survei Cyrus Network, Hasan Na...",Netral,
497,2022-12-25 14:18:20+00:00,Ciripaa_,"Pendiri lembaga survei Cyrus Network, Hasan Na...",Netral,
498,2022-12-25 14:11:25+00:00,KeceJomblo,"Pendiri lembaga survei Cyrus Network, Hasan Na...",Netral,
499,2022-12-25 12:52:05+00:00,Laskarorangbaik,Ganjar Calon Presiden 2024 #ganjarpranowo #gan...,Netral,papua barat


In [ ]:
tweets_df.to_csv('ganjar.csv', index=False)

## Crawling Data dari Portal Tugas Akhir Universitas Trunojoyo

In [ ]:
import requests
from bs4 import BeautifulSoup

pta = []

for i in range(10,61):
  URL = "https://pta.trunojoyo.ac.id/c_search/byprod/10/" + str(i+1)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  items = soup.find('div', {'id':'content_journal'})

  for it in items.findAll('li',{'data-cat':'#luxury'}):
    tempNama = it.span.text.strip('\n').replace('Penulis : ', '')
    # links = it.findAll('div', {'style':'margin: 25px 0px 15px 0px;'})
    link = it.find('a', 'gray button')['href']
    req = requests.get(link)
    soup2 = BeautifulSoup(req.text, 'html.parser')
    abstraks = soup2.findAll('div', {'style':'margin: 15px 15px 15px 15px;'})
    abstrak = soup2.find('p', {'align':'justify'}).text
  
    tempNIM = link.replace('https://pta.trunojoyo.ac.id/welcome/detail/', '')

    pta.append([it.a.text.strip('\n'), tempNIM, tempNama, abstrak])
    
pta_df = pd.DataFrame(pta, columns=['Judul', 'NIM', 'Penulis', 'Abstrak'])
pta_df

,Judul,NIM,Penulis,Abstrak
0,IMPLEMENTASI METODE N-GRAM PADA PEMBUATAN GAM...,080411100078,Farizal Arifin,Teknologi mobile game berkembang dengan sa...
1,IMPLEMENTASI PERFORMANCE PRISM \r\nUNTUK SISTE...,070411100081,Luhur Priyantoko Aji,Pengukuran kinerja adalah proses di mana organ...
2,PENGEMBANGAN VIRTUAL CLASS UNTUK PEMBELAJARAN ...,070411100042,Dwi Adi Permana,Internet berkembang dengan cepat beberapa tahu...
3,ANALISIS DAN PENERAPAN ALGORITMA PARTICLE SWAR...,070411100129,Qomarul Haryadi Irfan Rifa'i,Penjadwalan mata kuliah di program studi Tekni...
4,Rancang Bangun Sistem Pendukung Keputusan Pene...,080411100003,Ayu Winda Istara,Seiring dengan perubahan dalam paradigma pemba...
...,...,...,...,...
250,Pemetaan SMA/SMK/MA Di Kabupaten Lamongan Berd...,100411100054,Durorin Khumairoh,"Sarana dan prasarana yang memadai, jumlah lulu..."
251,REKOMENDASI PEMINJAMAN BUKU BERDASARKAN MINAT ...,100411100051,NAILY NAJIHAH,"Banyaknya keragaman buku di perpustakaan, meny..."
252,âVirtual Environment(VE) Kampus menggunakan ...,100411100026,Yusuf Machsar,Kampus merupakan lingkungan pendidikan perguru...
253,SISTEM PENDUKUNG KEPUTUSAN PENENTUAN PEMINATAN...,100411100079,Atin Hasanah,ABSTRAK \r\n\r\nTahun Ajaran 2013/2014 akan di...


In [ ]:
pta_df.to_csv('pta.csv', index=False)

## Crawling Data dari Portal Berita Online (Tempo)

In [7]:
import requests
from bs4 import BeautifulSoup

berita = []

URL = "https://www.tempo.co/indeks/2023-02-21/"
r = requests.get(URL)
  
soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find('main', 'main-left')

for it in items.findAll('h2','title'):
  judul = it.a.text
  link = it.a['href']

  req = requests.get(link)
    
  soup2 = BeautifulSoup(req.text, 'html.parser')
  contents = soup2.find('div', 'detail-in').text.strip('\nTEMPO.CO,')

  berita.append([judul, contents])
    
berita_df = pd.DataFrame(berita, columns=['Judul', 'Isi Berita'])
berita_df

,Judul,Isi Berita
0,"Keistimewaan 5 Masjid Terapung di Indonesia, S...",Jakarta - Kota Pariaman tahun ini akan memili...
1,6 Orang Tewas Akibat Gempa Susulan di Turki sa...,Jakarta - Enam orang tewas dalam gempa susula...
2,Begini Cara Memperoleh Izin Memiliki Senjata Api,Jakarta - Senjata api selama ini identik deng...
3,Momen Dramatis Evakuasi Udara Rombongan Kapold...,Jakarta - Momen evakuasi rombongan Kapolda Ja...
4,Liverpool vs Real Madrid di Liga Champions Sel...,Jakarta - Pertandingan Liverpool vs Real Madr...
...,...,...
300,Kejagung Banding atas Putusan Majelis Hakim PN...,Jakarta - Kejaksaan Agung Republik Indonesia ...
301,Duel Liverpool vs Real Madrid di Liga Champion...,Jakarta - Duel Liverpool vs Real Madrid bakal...
302,Pengamat Menilai Perluasan Kodam di Tiap Provi...,Jakarta -Rencana perluasan Komando Daerah Mil...
303,Saat Para Pesohor dan Fashionista Berkumpul Me...,Jakarta - Para selebriti dan fashionista berk...


In [8]:
berita_df.to_csv('berita.csv', index=False)